# NLI : Dataset SNLI

In [10]:
from datasets import load_dataset
import numpy as np
import pandas as pd

In [11]:
dataset = load_dataset("snli")

Found cached dataset snli (/Users/pepegarcia/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
100%|██████████| 3/3 [00:00<00:00, 415.35it/s]
